In [7]:
import datetime, pandas as pd

# Global locations of the raw data, and where to put the results
rawDataLocation = './RawData/'
resultsDataLocation = './ResultsData/'

# True to see print statements throughout
verbose = True

# Number of games to include; -1 for all; games are randomly selected
noOfGames = 45

# This unique number is appended to all of the results files
# If there is already a result file with this number, that section of the code
# is skipped
resultFilePrefix = '45Games';
#resultFilePrefix = '{date:%Y-%m-%d_%H.%M.%S}'.format(date=datetime.datetime.now())

# Files names for each of the steps; saveInterim will save steps 1 and 2
saveInterim = False
stepOneFileName = resultsDataLocation+resultFilePrefix+'_nfl_allTracking.csv'
stepTwoFileName = resultsDataLocation+resultFilePrefix+'_nfl_flat.csv'
stepThreeFileName = resultsDataLocation+resultFilePrefix+'_nfl_flatWithTeamUniquePlay.csv'

# The dataframe we're going to fill up
alltracking = pd.DataFrame()

In [8]:
import glob, random, os, numpy as np

# Step 1: concat all of the tracking files into a single file
# If there is a result file for this step, we can skip it or load it
if (os.path.isfile(stepOneFileName)):
    if (os.path.isfile(stepTwoFileName)):
        if (verbose): print('* Step 1 already exists, skipping : '+stepOneFileName)
    else:
        if (verbose): print('* Step 1 already exists, loading : '+stepOneFileName)
        alltracking = pd.read_csv(stepOneFileName)
        if (verbose): print(alltracking.info(verbose=True))
else:
    # Import all of the tracking csvs and concat them into a single data frame
    colnames = ['time_stamp','x','y','s','dis',
            'dir','event','nflId','displayName','jerseyNumber',
            'team','frame.id','gameId','playId']
    allcsvs = rawDataLocation + 'tracking_*.csv'

    # Get the glob of all of the files, and the random set of indexes
    globAllcsvs = sorted(glob.glob(allcsvs))
    if not (-1 < noOfGames < len(globAllcsvs)): noOfGames = len(globAllcsvs)
    randomGlobIndexes = random.sample(range(0, len(globAllcsvs)), noOfGames)

    # Look through and read the random files, add them to the dataframe
    if (verbose): print('Reading '+str(noOfGames)+' out of '+str(len(globAllcsvs))+' files')
    for index in range(len(randomGlobIndexes)):
        fileName = globAllcsvs[randomGlobIndexes[index]]
        if (verbose): print(' Read '+format(index+1, '03')+' : '+fileName)
        onefile = pd.read_csv(fileName, names=colnames, low_memory=False,
                              parse_dates=['time_stamp'], error_bad_lines=False)
        if (verbose): print('  - found '+str(len(onefile.index))+' records')
        alltracking = pd.concat([alltracking, onefile], ignore_index=False)

    # Get rid of the header rows
    if (verbose): print('Removing '
                        +str(len(alltracking)-len(alltracking[alltracking['x'] != 'x']))
                        +' header rows.')
    alltracking = alltracking[alltracking['x'] != 'x']

    # Get rid of any rows that don't have anything in the time_stamp column
    if (verbose): print('Removing '+str(len(alltracking)-alltracking['time_stamp'].count()
                                       +len(alltracking[alltracking['time_stamp'] == 'nan']))
                        +' rows with null timestamp.')
    alltracking = alltracking.dropna(subset=['time_stamp'])
    alltracking = alltracking[alltracking['time_stamp'] != 'nan']

    # Make sure all the columns are typed correctly.
    alltracking['x'] = alltracking['x'].astype(float);
    alltracking['y'] = alltracking['y'].astype(float);
    alltracking['s'] = alltracking['s'].astype(float);
    alltracking['dis'] = alltracking['dis'].astype(float);
    alltracking['dir'] = alltracking['dir'].astype(float);
    alltracking['event'] = alltracking['event'].astype(str);
    alltracking['nflId'] = alltracking['nflId'].fillna(-1).astype(int);
    alltracking['displayName'] = alltracking['displayName'].astype(str);
    alltracking['jerseyNumber'] = alltracking['jerseyNumber'].fillna(-1).astype(int);
    alltracking['team'] = alltracking['team'].astype(str);
    alltracking['frame.id'] = alltracking['frame.id'].astype(int);
    alltracking['gameId'] = alltracking['gameId'].astype(int);
    alltracking['playId'] = alltracking['playId'].astype(int);
    
    # Save the interim file
    if (saveInterim):
        print('Saving step 1 file : '+stepOneFileName)
        alltracking.to_csv(stepOneFileName)

    if (verbose): print(alltracking.info(verbose=True))

Reading 45 out of 91 files
 Read 001 : ./RawData/tracking_gameId_2017100105.csv
  - found 264363 records
 Read 002 : ./RawData/tracking_gameId_2017100800.csv
  - found 289368 records
 Read 003 : ./RawData/tracking_gameId_2017100803.csv
  - found 306867 records
 Read 004 : ./RawData/tracking_gameId_2017100104.csv
  - found 297029 records
 Read 005 : ./RawData/tracking_gameId_2017091003.csv
  - found 247642 records
 Read 006 : ./RawData/tracking_gameId_2017092500.csv
  - found 258448 records
 Read 007 : ./RawData/tracking_gameId_2017092800.csv
  - found 250425 records
 Read 008 : ./RawData/tracking_gameId_2017092400.csv
  - found 257406 records
 Read 009 : ./RawData/tracking_gameId_2017092402.csv
  - found 242053 records
 Read 010 : ./RawData/tracking_gameId_2017101508.csv
  - found 283950 records
 Read 011 : ./RawData/tracking_gameId_2017100809.csv
  - found 247270 records
 Read 012 : ./RawData/tracking_gameId_2017091007.csv
  - found 259508 records
 Read 013 : ./RawData/tracking_gameId

In [9]:
from datetime import time

# Step 2: Flatten out by adding the other tables to alltracking
# If there is a result file for this step, we can skip it or load it
if (os.path.isfile(stepTwoFileName)):
    if (os.path.isfile(stepThreeFileName)):
        if (verbose): print('* Step 2 already exists, skipping : '+stepTwoFileName)
    else:
        if (verbose): print('* Step 2 already exists, loading : '+stepTwoFileName)
        alltracking = pd.read_csv(stepTwoFileName)
        if (verbose): print(alltracking.info(verbose=True))

else:
    # First add in the plays data, parsing the time to be minutes:seconds
    def timeparse (timestamp_str):
        timeparsetimes = timestamp_str.split(":")
        return time(0, int(timeparsetimes[0]), int(timeparsetimes[1]))
    playsFile = rawDataLocation + 'plays.csv'
    if (verbose): print('Read plays file : '+playsFile)
    plays = pd.read_csv(playsFile, parse_dates=['GameClock'], date_parser=timeparse)
    if (verbose): print('  - found '+str(len(plays.index))+' records')
    alltracking = pd.merge(alltracking, plays, on=['playId','gameId'], how='left')
    if (verbose): print('Merged plays with '+str(len(alltracking.index))+' tracking records')

    # Next add the games data
    gamesFile = rawDataLocation + 'games.csv'
    if (verbose): print('Read games file : '+gamesFile)
    games = pd.read_csv(gamesFile, parse_dates=['gameDate'])
    if (verbose): print('  - found '+str(len(games.index))+' records')
    games['gameDate'] = pd.DatetimeIndex(games['gameDate']).date
    alltracking = pd.merge(alltracking, games, on=['gameId'], how='left')
    if (verbose): print('Merged games with '+str(len(alltracking.index))+' tracking records')

    # Lastly, add the player data
    playersFile = rawDataLocation + 'players.csv'
    if (verbose): print('Read players file : '+gamesFile)
    players = pd.read_csv(playersFile)
    if (verbose): print('  - found '+str(len(players.index))+' records')
    alltracking = pd.merge(alltracking, players, on=['nflId'], how='left')
    if (verbose): print('Merged players with '+str(len(alltracking.index))+' tracking records')

    # Save the interim file
    if (saveInterim):
        print('Saving step 2 file : '+stepTwoFileName)
        alltracking.to_csv(stepTwoFileName)

    if (verbose): print(alltracking.info(verbose=True))

Read plays file : ./RawData/plays.csv
  - found 14193 records
Merged plays with 12356010 tracking records
Read games file : ./RawData/games.csv
  - found 91 records
Merged games with 12356010 tracking records
Read players file : ./RawData/games.csv
  - found 1713 records
Merged players with 12356010 tracking records
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12356010 entries, 0 to 12356009
Data columns (total 68 columns):
time_stamp                object
x                         float64
y                         float64
s                         float64
dis                       float64
dir                       float64
event                     object
nflId                     int64
displayName               object
jerseyNumber              int64
team                      object
frame.id                  int64
gameId                    int64
playId                    int64
quarter                   int64
GameClock                 object
down                      int64
yardsToG

In [10]:
from datetime import datetime, timedelta, time

# Step 3: Add the game clock, and the unique play index
# If there is a result file for this step, we can skip it or load it
if (os.path.isfile(stepThreeFileName)):
    if (verbose): print('* Step 3 already exists, skipping : '+stepThreeFileName)

else:
    # Add a team column to every tracking row, based on home or away team
    alltracking['teamName'] = np.where(alltracking['team'] == 'away',
                                       alltracking['visitorDisplayName'], np.NaN)
    alltracking['teamName'] = np.where(alltracking['team'] == 'home',
                                       alltracking['homeDisplayName'], alltracking['teamName'])
    if (verbose): print('Added team name to '+str(len(alltracking.index))+' tracking records')

    # Add a column to give every play a unique id (combination of gameId and playId)
    alltracking['uniquePlayId'] = alltracking.apply(lambda row:
                                            int(str(row['gameId'])+
                                            format(row['playId'], '04')),
                                            axis=1)
    if (verbose): print('Added unique play to '+str(len(alltracking.index))
                        +' tracking records')

    # Add a column for full game time that has the game clock for the game, not the qtr
    def add_qtr_minutes (row):
        addMinutes = 0
        if (1 < row['quarter'] < 5 ):
            addMinutes = (row['quarter'] - 1) * 15
        if (row['quarter'] > 4):
            addMinutes = 45 + ((row['quarter'] - 5) * 10)

        rowGameClock = datetime.strptime(str(row['GameClock']), '%H:%M:%S').time()
        newTotalGameTime = datetime.combine(datetime.today(), time(0,15,0))
        newTotalGameTime -= timedelta(minutes=rowGameClock.minute, seconds=rowGameClock.second)
        newTotalGameTime += timedelta(minutes=addMinutes)
        return newTotalGameTime.time()
    alltracking['FullGameTime'] = alltracking.apply(lambda row: add_qtr_minutes(row), axis=1)
    if (verbose): print('Added full game clock to '+str(len(alltracking.index))
                        +' tracking records')

    # Add a new column with the game date and game clock
    def add_quarter_minutes_with_date (row):
        rowTimeStamp = datetime.strptime(row['time_stamp'], '%Y-%m-%dT%H:%M:%SZ').date()
        return str(datetime.combine(rowTimeStamp, row['FullGameTime']))

    alltracking['FullGameTimeDate'] = alltracking.apply(lambda row:
                                                  add_quarter_minutes_with_date(row), axis=1)

    # Save the final file
    print('Saving step 3 file : '+stepThreeFileName)
    alltracking.to_csv(stepThreeFileName)

    if (verbose): print(alltracking.info(verbose=True))

Added team name to 12356010 tracking records
Added unique play to 12356010 tracking records
Added full game clock to 12356010 tracking records
Saving step 3 file : ./ResultsData/45Games_nfl_flatWithTeamUniquePlay.csv
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12356010 entries, 0 to 12356009
Data columns (total 72 columns):
time_stamp                object
x                         float64
y                         float64
s                         float64
dis                       float64
dir                       float64
event                     object
nflId                     int64
displayName               object
jerseyNumber              int64
team                      object
frame.id                  int64
gameId                    int64
playId                    int64
quarter                   int64
GameClock                 object
down                      int64
yardsToGo                 int64
possessionTeam            object
yardlineSide              object
yardlineNum